In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
import sys

# Add the parent directory to the Python path
script_dir = Path().resolve() # Get the current working directory
sys.path.append(str(script_dir.parent))
import src.config as config 

In [3]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')

current_date=Timestamp('2024-07-13 01:00:00')


C:\Users\DELL\AppData\Local\Temp\ipykernel_17140\842599459.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)


In [6]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/840852
Connected. Call `.close()` to terminate connection gracefully.
fetching data from 2024-06-15 01:00:00+00:00 to 2024-07-13 00:00:00+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (11.92s) 


In [7]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/840852
Connected. Call `.close()` to terminate connection gracefully.
[LightGBM] [Warning] feature_fraction is set=0.866100376126804, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.866100376126804
[LightGBM] [Warning] bagging_fraction is set=0.5951041299254337, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5951041299254337


In [ ]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,9.0,2024-07-10 21:00:00
1,2,6.0,2024-07-10 21:00:00
2,3,6.0,2024-07-10 21:00:00
3,4,9.0,2024-07-10 21:00:00
4,5,6.0,2024-07-10 21:00:00
...,...,...,...
260,261,3.0,2024-07-10 21:00:00
261,262,6.0,2024-07-10 21:00:00
262,263,3.0,2024-07-10 21:00:00
263,264,4.0,2024-07-10 21:00:00


In [8]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/840852
Connected. Call `.close()` to terminate connection gracefully.


In [12]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

FeatureStoreException: Features are not compatible with Feature Group schema: 
 - pickup_hour (type: 'timestamp') is missing from input dataframe.
Note that feature (or column) names are case insensitive and spaces are automatically replaced with underscores.